### Import libraries

In [1]:
import cv2
import numpy as np
import pandas as pd
# from absl import flags
import tensorflow as tf
import tensorflow_privacy
from keras.models import Model
import matplotlib.pyplot as plt
from skimage.feature import hog
from PIL import Image, ImageOps
import tensorflow_addons as tfa
from keras.models import Sequential, Model
# from livelossplot import PlotLossesKeras
from keras.applications.vgg16 import VGG16
from keras.layers.convolutional import Conv2D
from tensorflow.keras.datasets import cifar10
from tensorflow_addons.layers import GroupNormalization
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy
from tensorflow_privacy.privacy.analysis.rdp_accountant import compute_rdp
from keras.layers import Input, Dense, Flatten, Dropout, Add, LayerNormalization
from tensorflow_privacy.privacy.analysis.rdp_accountant import get_privacy_spent
from keras.layers.pooling import MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
from tensorflow_privacy import DPKerasSGDOptimizer, DPKerasAdamOptimizer, DPKerasAdagradOptimizer
tf.compat.v1.disable_v2_behavior()
tf.get_logger().setLevel('ERROR')
%matplotlib inline

Instructions for updating:
non-resource variables are not supported in the long term


### Load Dataset

In [2]:
def hog_transform(train):
    train_hog = []
    for sample in train:
        img = Image.fromarray(np.uint8(sample)).convert('RGB')
        img = ImageOps.grayscale(img)
        hog_features = hog(img, orientations = 9, pixels_per_cell = (8, 8), 
                           cells_per_block = (4, 4), block_norm = 'L2', visualize = False)
        train_hog.append(hog_features)
    train_hog = np.array(train_hog)
    return train_hog

In [3]:
def resize_data(train, newsize):
    train_resized = []
    for sample in train:
        img = Image.fromarray(np.uint8(sample)).convert('RGB').resize(newsize)
        train_resized.append(np.array(img))
    train_resized = np.array(train_resized)
    return train_resized

In [4]:
# Loading cifar10 image dataset
data_train, data_test = cifar10.load_data()
x_train, y_train = data_train
x_test, y_test = data_test

# # Hog transformations of images
# x_train_hog = hog_transform(x_train)
# x_test_hog = hog_transform(x_test)

# Resize images
x_train = resize_data(x_train, (128, 128))
x_test = resize_data(x_test, (128, 128))

# Normalizing pixel values of images
x_train = x_train / 255.0
x_test = x_test / 255.0

# Flattening images in dataset
y_train = y_train.flatten().reshape(y_train.shape[0], 1)
y_test = y_test.flatten().reshape(y_test.shape[0], 1)

# One hot encoding of labels/target column
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

print("training:", x_train.shape, y_train.shape)
print("testing:", x_test.shape, y_test.shape)
print("--------------------------------")
# print("hog training:", x_train_hog.shape, y_train.shape)
# print("hog testing:", x_test_hog.shape, y_test.shape)

training: (50000, 128, 128, 3) (50000, 10)
testing: (10000, 128, 128, 3) (10000, 10)
--------------------------------


### Model Architectures

In [5]:
#### VGG with transfer learning
def pre_trained_vgg():
    #Building a VGG model with pretrained weights from cifar100
    model = Sequential()
    vgg_model = VGG16(include_top=False, weights="imagenet",
                      classes=10, pooling=max, input_shape=(32,32,3))
    vgg_model.trainable=False
    model.add(vgg_model)
    model.add(Flatten())
    model.add(Dense(8, activation='tanh'))
    model.add(Dense(16, activation='tanh'))
    model.add(Dense(10))
    return model


#### HOG + ANN
def hog_ann():
    ### Feed-forward network
    model_input = Input(shape = (144))
    d = Dense(32, activation = 'tanh')(model_input)
    d = LayerNormalization()(d)
    d = Dropout(0.4)(d)
    d = Dense(16, activation = 'tanh')(d)
    d = LayerNormalization()(d)
    d = Dense(16, activation = 'tanh')(d)
    d = Dropout(0.4)(d)
    d = Dense(8, activation = 'tanh')(d)
    d = LayerNormalization()(d)
#     d = Dense(8, activation = 'tanh')(d)
    model_output = Dense(10)(d)
    model = Model(inputs = model_input, outputs = model_output)
    return model


#### Custom CNN
def cnn():
    ## CNN network
    model_input = Input(shape = (64, 64, 3))
    x1 = Conv2D(32, (8, 8), activation = 'tanh', strides = 2)(model_input)
    x1 = LayerNormalization()(x1)
    x1 = Dropout(0.4)(x1)
    x1 = MaxPooling2D(pool_size = (2,1))(x1)
    x2 = Conv2D(32, (4, 4), activation = 'tanh', strides = 2)(x1)
    x2 = MaxPooling2D(pool_size = (2,1))(x2)
    x2 = LayerNormalization()(x2)
    ftn = Flatten()(x2)
    ### Feed-forward network
    d1 = Dense(16, activation = 'tanh')(ftn)
    d1 = Dropout(0.1)(d1)
    d2 = Dense(16, activation = 'tanh')(d1)
    d2 = LayerNormalization()(d2)
    d3 = Dropout(0.1)(d2)
    d4 = Dense(8, activation = 'tanh')(d3)
    d4 = LayerNormalization()(d4)
    d5 = Dense(8, activation = 'tanh')(d4)
    model_output = Dense(10, activation = 'softmax')(d5)
    model = Model(inputs = model_input, outputs = model_output)
    return model


#### Pretrained VGG with custom architecture (ResNet)
def custommodel():
#     vgg_model = VGG16(include_top=False, weights="imagenet", 
#                   classes=10, pooling=max, input_shape=(128,128,3))
#     # re-structure the model
#     vgg_model = Model(inputs=vgg_model.inputs, outputs=vgg_model.layers[5].output)
#     vgg_model.trainable=False
    model_input = Input(shape = (128, 128, 3))
#     vgg_output = vgg_model(model_input, training=False)
    x1 = Conv2D(16, (3, 3), activation = 'tanh', strides = 2)(model_input)
#     x2 = Conv2D(8, (3, 3), activation = 'tanh', strides = 2)(x1)
    x2 = MaxPooling2D(pool_size = (3,3))(x1)
    ## Residual Block 1
    x3 = Conv2D(16, (3, 3), activation = 'tanh', strides = 1)(x2)
    x3 = LayerNormalization()(x3) # normalization layer 1
    x4 = Conv2D(16, (3, 3), activation = 'tanh', strides = 1)(x3)
#     sk1 = Add()([x2, x4]) # skip connection 1
    ## Residual Block 2
    x5 = Conv2D(16, (3, 3), activation = 'tanh', strides = 1)(x4)
    x5 = LayerNormalization()(x5) # normalization layer 2
    x6 = Conv2D(16, (3, 3), activation = 'tanh', strides = 1)(x5)
#     sk2 = Add()([sk1, x6]) # skip connection 2
    ## Residual Block 3
    x7 = Conv2D(16, (3, 3), activation = 'tanh', strides = 1)(x6)
    x7 = LayerNormalization()(x7) # normalization layer 3
    x8 = Conv2D(16, (3, 3), activation = 'tanh', strides = 1)(x7)
#     sk3 = Add()([sk2, x8]) # skip connection 3
    ## Residual Block 4
    x9 = Conv2D(16, (3, 3), activation = 'tanh', strides = 1)(x8)
    x9 = LayerNormalization()(x9) # normalization layer 4
    x10 = Conv2D(16, (3, 3), activation = 'tanh', strides = 1)(x9)
#     sk4 = Add()([sk3, x10]) # skip connection 4
    ## Global Average Pooling layer
    avg = GlobalAveragePooling2D()(x10)
    ftn = Flatten()(avg)
    ## Feed-forward network
    d1 = Dense(16, activation = 'tanh')(ftn)
    d1 = Dropout(0.4)(d1)
    d2 = Dense(16, activation = 'tanh')(d1)
    d2 = Dropout(0.4)(d2)
    model_output = Dense(10)(d2)
    model = Model(inputs = model_input, outputs = model_output)
    return model


#### ResNet-10
def resnet():
    ### Custom ResNet model
    model_input = Input(shape = (32, 32, 3))
    x1 = Conv2D(64, (3, 3), activation = 'tanh', strides = 2)(model_input)
#     x2 = Conv2D(32, (3, 3), activation = 'tanh', strides = 2)(x1)
    x2 = MaxPooling2D(pool_size = (3,3))(x1)
    ## Residual Block 1
    x3 = Conv2D(64, (3, 3), activation = 'relu', strides = 1)(x2)
    x3 = LayerNormalization()(x3) # normalization layer 1
    x4 = Conv2D(64, (3, 3), activation = 'relu', strides = 1)(x3)
    sk1 = Add()([x2, x4]) # skip connection 1
    ## Residual Block 2
    x5 = Conv2D(64, (3, 3), activation = 'relu', strides = 1)(sk1)
    x5 = LayerNormalization()(x5) # normalization layer 2
    x6 = Conv2D(64, (3, 3), activation = 'relu', strides = 1)(x5)
    sk2 = Add()([sk1, x6]) # skip connection 2
    ## Residual Block 3
    x7 = Conv2D(64, (3, 3), activation = 'relu', strides = 1)(sk2)
    x7 = LayerNormalization()(x7) # normalization layer 3
    x8 = Conv2D(64, (3, 3), activation = 'relu', strides = 1)(x7)
    sk3 = Add()([sk2, x8]) # skip connection 3
    ## Residual Block 4
    x9 = Conv2D(64, (3, 3), activation = 'relu', strides = 1)(sk3)
    x9 = LayerNormalization()(x9) # normalization layer 4
    x10 = Conv2D(64, (3, 3), activation = 'relu', strides = 1)(x9)
    sk4 = Add()([sk3, x10]) # skip connection 4
    ## Global Average Pooling layer
    avg = GlobalAveragePooling2D()(sk4)
    ftn = Flatten()(avg)
    # Feed-forward network
    d1 = Dense(16, activation = 'tanh')(ftn)
    d2 = Dense(16, activation = 'tanh')(d1)
    model_output = Dense(10)(d2)
    model = Model(inputs = model_input, outputs = model_output)
    return model


#### VGG-8
def vgg():
    ### Custom VGG model
    model_input = Input(shape = (32, 32, 3))
    x1 = Conv2D(8, (3, 3), activation = 'tanh', strides = 1)(model_input)
#     x2 = Conv2D(64, (3, 3), activation = 'tanh', strides = 1)(x1)
    x2 = MaxPooling2D(pool_size = (3,3), strides = 2)(x1)
    x3 = Conv2D(16, (3, 3), activation = 'tanh', strides = 1)(x2)
    x4 = Conv2D(32, (3, 3), activation = 'tanh', strides = 1)(x3)
    x4 = MaxPooling2D(pool_size = (3,3), strides = 2)(x4)
#     x5 = Conv2D(32, (3, 3), activation = 'tanh', strides = 1)(x4)
#     x6 = Conv2D(32, (3, 3), activation = 'relu', strides = 1)(x5)
#     x7 = Conv2D(32, (3, 3), activation = 'relu', strides = 1)(x6)
    x7 = MaxPooling2D(pool_size = (3,3), strides = 2)(x4)
    ftn = Flatten()(x7)
    # Feed-forward network
    d1 = Dense(8, activation = 'tanh')(ftn)
    d1 = Dropout(0.4)(d1)
    d2 = Dense(16, activation = 'tanh')(d1)
    d2 = Dropout(0.4)(d2)
    model_output = Dense(10)(d2)
    model = Model(inputs = model_input, outputs = model_output)
    return model

### Privacy Hyper-parameters

In [6]:
epochs = 100 
batch_size = 250 
l2_norm_clip = 1 
noise_multiplier = 1.2
num_microbatches = batch_size
learning_rate = 0.15   
delta = 1e-5

if batch_size % num_microbatches != 0:
    raise ValueError('Batch size should be an integer multiple of the number of microbatches')
    
# Compute RDP
orders = [1 + x / 10. for x in range(1, 100)] + list(range(12, 64))
rdp = compute_rdp(q = batch_size / 50000,
                  noise_multiplier = noise_multiplier,
                  steps = epochs * 50000 // batch_size,
                  orders = orders)
# Calculate epsilon
epsilon = get_privacy_spent(orders, rdp, target_delta = delta)[0]
epsilon

3.2560909204328192

### Optimizer and Loss function

In [7]:
# Create optimizer.

### Time-based learning rate decay
decay = learning_rate / epochs
# def lr_time_based_decay(epoch, lr):
#     return lr * 1 / (1 + decay * epoch)

# define optimizer (dp-sgd) 
optimizer = DPKerasSGDOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=num_microbatches,
    learning_rate=learning_rate,
    decay=decay)

# # define optimizer (dp-adam)
# optimizer = DPKerasAdamOptimizer(
#     l2_norm_clip=l2_norm_clip,
#     noise_multiplier=noise_multiplier,
#     num_microbatches=num_microbatches,
#     learning_rate=learning_rate)

# # define optimizer (dp-adagrad)
# optimizer = DPKerasAdagradOptimizer(
#     l2_norm_clip=l2_norm_clip,
#     noise_multiplier=noise_multiplier,
#     num_microbatches=num_microbatches,
#     learning_rate=learning_rate)

# Compute loss
loss = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True, reduction=tf.losses.Reduction.NONE)

### Compile and train model

In [8]:
model_hog = False
## Model
# model = pre_trained_vgg()
# model_hog = hog_ann()
model = custommodel()
# model = cnn()
# model = resnet()
# model = vgg()

## HOG_training
if model_hog:
    x_train = x_train_hog
    x_test = x_test_hog 
    model = model_hog
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 63, 63, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 21, 21, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 19, 19, 16)        2320      
                                                                 
 layer_normalization (LayerN  (None, 19, 19, 16)       32        
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 17, 17, 16)        2320  

In [ ]:
## Compile model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
## Train model
history = model.fit(x_train, y_train, epochs=epochs, validation_data=(x_test, y_test),
#                     callbacks=[LearningRateScheduler(lr_time_based_decay, verbose=1)],
                    batch_size=batch_size)

Train on 50000 samples, validate on 10000 samples
Metal device set to: Apple M1 Pro


2022-04-06 17:39:34.760709: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-06 17:39:34.760852: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-04-06 17:39:36.268273: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-04-06 17:39:36.293864: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-06 17:39:37.786442: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Epoch 1/100


2022-04-06 17:39:38.138614: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-06 17:39:38.776565: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


17000/50000 [=========>....................] - ETA: 2:23:02 - loss: 2.3802 - acc: 0.1089

In [ ]:
### Plotting accuracy/loss graph
def get_loss(history_loss):
    loss = []
    for epoch in history_loss:
        loss.append(epoch[-1])
    return loss
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = get_loss(history.history['loss'])
val_loss = get_loss(history.history['val_loss'])
print('train_acc:', round(acc[-1], 3), "    ", 'train_loss:', round(loss[-1], 3))
print('val_acc:', round(val_acc[-1], 3), "      ", 'val_loss:', round(val_loss[-1], 3))
fig = plt.figure(figsize=(15, 5))
## Accuracy plot
plt.subplot(1, 2, 1) 
plt.plot(acc)
plt.plot(val_acc)
plt.title('Accuracy')
plt.ylabel('model_accuracy')
plt.xlabel('epoch')
## Loss plot
plt.subplot(1, 2, 2) 
plt.plot(loss)
plt.plot(val_loss)
plt.title('Loss')
plt.ylabel('model_loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()